# Daily Stopwatch Data Science: Play around with SEC Data

Note: this is a part of understandin SEC data. See https://www.sec.gov/dera/data/financial-statement-data-sets.html

## Stage 1: Ask a question

My objective is to extract financial statements from the company that I am interested. Say Southwest.

There is no performance measure for the visualization yet.

## Stage 2: Set the environment up and get data

First, set up a directory for data and link it to this workplace. Download data into your choice of directory.

In [ ]:
#Set up the environment
import pandas as pd                        #Pandas
import numpy as np                         #Numpy

### With large folders

In [75]:
# Set up data directory
DataDir = "C:/Users/Admin/Downloads/2017q1/"

# Set up output directory
#OutputDir = "C:/Users/Admin/Documents/GitHub/KhanAcademyThailand/"

#Read data
#df = pd.pandas.read_excel(DataDir+filename,sheetname='AllAvailableThaiVideos')

There are four files: sub, tag, num, pre (See readme in the folder for more info.). 

First, let's look at the sub table. There are 36 columns

In [132]:
filename = "sub.txt"
with open(DataDir+filename) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]

In [133]:
sub = pd.DataFrame(columns = content[0].split('\t'))

In [134]:
for i in range(0,36):
    sub[sub.columns[i]] = map(lambda x: (x.split('\t')+(36-len(x.split('\t')))*[''])[i],content[1:])

In [135]:
len(sub)

6270

Create a look up system based on CIK. Use website: https://www.sec.gov/edgar/searchedgar/cik.htm
For example Southwest Airline CIK is 92380. 

In [136]:
sub[sub['cik']=='92380']

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,...,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
291,0000092380-17-000029,92380,SOUTHWEST AIRLINES CO,4512,US,TX,DALLAS,75235,2702 LOVE FIELD DR,P O BOX 36611,...,20161231,2016,FY,20170207,2017-02-07 16:35:00.0,0,1,luv-20161231.xml,1,


In [137]:
sub.loc[291]

adsh           0000092380-17-000029
cik                           92380
name          SOUTHWEST AIRLINES CO
sic                            4512
countryba                        US
stprba                           TX
cityba                       DALLAS
zipba                         75235
bas1             2702 LOVE FIELD DR
bas2                  P O BOX 36611
baph                     2147924000
countryma                        US
stprma                           TX
cityma                       DALLAS
zipma                    75235-1611
mas1                   PO BOX 36611
mas2                               
countryinc                       US
stprinc                          TX
ein                       741563240
former             AIR SOUTHWEST CO
changed                    19760108
afs                           1-LAF
wksi                              1
fye                            1231
form                           10-K
period                     20161231
fy                          

Here there is a column called 'form'. Let's check it out.

In [138]:
set(sub['form'])

{'10-K',
 '10-K/A',
 '10-KT',
 '10-KT/A',
 '10-Q',
 '10-Q/A',
 '10-QT',
 '20-F',
 '20-F/A',
 '40-F',
 '40-F/A',
 '424B5',
 '6-K',
 '6-K/A',
 '8-K',
 '8-K/A',
 'POS AM',
 'S-1',
 'S-1/A',
 'S-11/A',
 'S-4',
 'S-4/A'}

We're only interested in Quarter Report (10-Q). Let's filter it out along with the company code.

In [141]:
sub = sub[((sub['form'] =='10-Q')|(sub['form']=='10-Q/A'))&(sub['cik']=='92380')]

In [142]:
len(sub)

0

Not update!? :(

Next, let's look at the tag table. There are 9 columns

In [104]:
filename = "tag.txt"
with open(DataDir+filename) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]

In [106]:
tag = pd.DataFrame(columns = content[0].split('\t'))

In [107]:
for i in range(0,9):
    tag[tag.columns[i]] = map(lambda x: (x.split('\t')+(9-len(x.split('\t')))*[''])[i],content[1:])

In [38]:
filename = "num.txt"
with open(DataDir+filename) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]

In [119]:
tag[:5]

,tag,version,custom,abstract,datatype,iord,crdr,tlabel,doc
0,AccountsPayableCurrent,us-gaap/2015,0,0,monetary,I,C,"Accounts Payable, Current",Carrying value as of the balance sheet date of...
1,AccountsPayableRelatedPartiesCurrent,us-gaap/2015,0,0,monetary,I,C,"Accounts Payable, Related Parties, Current",Amount for accounts payable to related parties...
2,AccountsReceivableNetCurrent,us-gaap/2015,0,0,monetary,I,D,"Accounts Receivable, Net, Current","Amount due from customers or clients, within o..."
3,AccruedLiabilitiesCurrent,us-gaap/2015,0,0,monetary,I,C,"Accrued Liabilities, Current",Carrying value as of the balance sheet date of...
4,AccumulatedDepreciationDepletionAndAmortizatio...,us-gaap/2015,0,0,monetary,I,C,"Accumulated Depreciation, Depletion and Amorti...","Amount of accumulated depreciation, depletion ..."


In [124]:
len(set(tag['tag']))

61659

In [110]:
len(tag)

81519

If we understand these tags, we can recover relevant quantities we want.

Next, let's look at num table. There are 9 columns

In [78]:
num = pd.DataFrame(columns = content[0].split('\t'))

In [83]:
for i in range(0,9):
    num[num.columns[i]] = map(lambda x: (x.split('\t')+(9-len(x.split('\t')))*[''])[i],content[1:])

In [84]:
num[:5]

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
0,0001477932-16-014447,EntityCommonStockSharesOutstanding,dei/2014,,20161231,0,shares,67538142.0000,
1,0001477932-16-014447,AccountsAndOtherReceivablesNetCurrent,us-gaap/2016,,20151231,0,USD,,
2,0001477932-16-014447,AccountsAndOtherReceivablesNetCurrent,us-gaap/2016,,20160930,0,USD,,
3,0001477932-16-014447,AccountsReceivableNetCurrent,us-gaap/2016,,20151231,0,USD,95563.0000,
4,0001477932-16-014447,AccountsReceivableNetCurrent,us-gaap/2016,,20160930,0,USD,146102.0000,


In [89]:
#Check out unique tags
len(set(num['tag'])), len(set(num['adsh']))

(46560, 6269)

Finally, let's look at the pre table. There are 10 columns

In [112]:
filename = "pre.txt"
with open(DataDir+filename) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]

In [113]:
pre = pd.DataFrame(columns = content[0].split('\t'))

In [114]:
for i in range(0,10):
    pre[pre.columns[i]] = map(lambda x: (x.split('\t')+(10-len(x.split('\t')))*[''])[i],content[1:])

In [115]:
pre[:5]

,adsh,report,line,stmt,inpth,rfile,tag,version,plabel,negating
0,0000804753-17-000004,2,20,BS,0,H,AccountsPayableCurrent,us-gaap/2015,Accounts payable,0
1,0000215466-17-000058,5,19,BS,0,H,AccountsPayableCurrent,us-gaap/2015,Accounts payable,0
2,0001477932-17-000685,2,10,BS,0,H,AccountsPayableCurrent,us-gaap/2015,Accounts payable,0
3,0001554795-17-000056,2,11,BS,0,H,AccountsPayableCurrent,us-gaap/2015,Accounts payable,0
4,0001558370-17-000547,3,19,BS,0,H,AccountsPayableCurrent,us-gaap/2015,Accounts payable,0


In [116]:
len(pre)

1008611

## Stage 3: What's next?

It seems that this data set is difficult to work with. Let's instead focus on individual financial statements. See the next book.

*Note to myself:* I haven't tracked Pomodoros. I didn't get the flow but OK. Let's focus more in the next round.

In [9]:
https://www.sec.gov/Archives/edgar/data/{cik}/{adsh without dash}/

In [ ]:
https://www.sec.gov/Archives/edgar/data/92380/000009238017000078/Financial_Report.xlsx